In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
from torch.utils.data import TensorDataset
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights   
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score

batch_size = 128
num_workers = 8

In [6]:

# Load the saved fold data from the .pkl file
fold_data = torch.load('/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/cross_val_data/fold_data.pt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold_info in fold_data:
    fold = fold_info['fold']
    print("Fold: ", fold)
    X_train = fold_info['X_train']
    y_train = fold_info['y_train']
    X_test = fold_info['X_test']
    y_test = fold_info['y_test']
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoader for training and testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create and train the model
    #print datatype of trainloader
    print("trainloader datatype: ", train_loader.dataset.tensors[1].dtype)
    print("testloader datatype: ", test_loader.dataset.tensors[0].dtype)
    train_model = EfficientNetClassifier    (
        models.efficientnet_b0, EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
    ).to(device)

    iter_losses, epoch_losses = train_fn(
        train_model,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        lr=3e-4,
        epochs=5,
        verbose=True,
        wandb_log=False,
    )

    # Evaluate the model on the test set
    with torch.no_grad():
        pred_classes = train_model.predict_class(
            dataloader=test_loader, batch_size=batch_size, verbose=True
        ).to(device)

    test_labels = torch.from_numpy(y_test).to(device)
    # Calculate and print metrics for each fold
    
    accuracy = accuracy_score(test_labels, pred_classes)
    precision = precision_score(test_labels, pred_classes)
    recall = recall_score(test_labels, pred_classes)
    f1 = f1_score(test_labels, pred_classes)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")
    # Calculate and store metrics for each fold
    accuracy_list.append(accuracy_score(test_labels, pred_classes))
    precision_list.append(precision_score(test_labels, pred_classes))
    recall_list.append(recall_score(test_labels, pred_classes))
    f1_list.append(f1_score(test_labels, pred_classes))

# Calculate and print the mean of metrics across all folds
print("Mean Accuracy: ", sum(accuracy_list) / len(accuracy_list))
print("Mean Precision: ", sum(precision_list) / len(precision_list))
print("Mean Recall: ", sum(recall_list) / len(recall_list))
print("Mean F1: ", sum(f1_list) / len(f1_list))


Fold:  1
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


Loss: 0.00413392: 100%|██████████| 5/5 [02:59<00:00, 35.96s/it]


Fold:  2
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


Loss: 0.00555695: 100%|██████████| 5/5 [02:43<00:00, 32.77s/it]


Fold:  3
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


Loss: 0.00667962: 100%|██████████| 5/5 [09:57<00:00, 119.59s/it]


Fold:  4
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


Loss: 0.00590688: 100%|██████████| 5/5 [03:26<00:00, 41.39s/it] 


: 